In [1]:
# Importação as bibliotecas necessárias para conexão e coleta dos dados
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
# Variaveis ambientes

host = os.environ.get('HOST')
user = os.environ.get('USER')
password = os.environ.get('PASSWORD')

In [3]:
# Descrevo os parâmetros de conexão conforme configurado
db_server_info = {'host': host, 
                  'porta': 5432, 
                  'dbname': 'dbs_vacina',
                  'user': user, 
                  'password': password} 

conn_str = '{}://{}:{}@{}:{}/{}'.format('postgresql+psycopg2', 
                                  db_server_info['user'], 
                                  db_server_info['password'], 
                                  db_server_info['host'], 
                                  db_server_info['porta'], 
                                  db_server_info['dbname']) 

alchemyEngine = create_engine(conn_str, pool_recycle=3600)

postgreSQLConnection = alchemyEngine.connect() 

In [4]:
# Filtrando a tbl_cidade por UF = DF

pd.read_sql_query("SELECT * FROM pos.tbl_cidade WHERE uf = 'DF'", postgreSQLConnection)

,id,cd_ibge_reduzido,nm_municipio,uf,lat,lon
0,5300108,530010,Brasília,DF,-15.791295,-47.883132


In [5]:
# Filtrando a tbl_cep pelo id de Brasília (5300108)

df_ceps_df = pd.read_sql_query("SELECT * FROM pos.tbl_cep WHERE id_cidade = 5300108", postgreSQLConnection)
df_ceps_df.head()

,id,id_cidade,lat,lon
0,70002,5300108,-15.790840,-47.879656
1,70003,5300108,-15.790649,-47.880634
2,70037,5300108,-15.805816,-47.882790
3,70040,5300108,-15.791919,-47.884855
4,70041,5300108,-15.788176,-47.881767


In [6]:
# Filtrando a tbl_estabelecimento pelos cep de Brasília

ceps = df_ceps_df.id

df_estab = pd.read_sql_query("SELECT * FROM pos.tbl_estabelecimento", postgreSQLConnection)

df_estab_df = df_estab.loc[df_estab['id_cep'].isin(ceps)].copy()
df_estab_df.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,id_tipo_estabelecimento,id_atividade_principal,lat,lon
275936,3266303,RONILDA SAUL HAAS,RONILDA SAUL HAAS,70100.0,4000.0,NaN,NaN,NaN,NaN
276064,3106632,FLAVIA DE PASSOS,FLAVIA DE PASSOS,70100.0,4000.0,NaN,NaN,NaN,NaN
276308,3464857,CLINICA CONIC LTDA,ECOGRAFIAS CONIC,70100.0,2000.0,NaN,NaN,NaN,NaN
277135,7966946,LIFE ODONTOLOGIA LTDA,LIFE ODONTOLOGIA,70200.0,2062.0,NaN,NaN,NaN,NaN
277573,3522032,STOMATUS ODONTOLOGIA LTDA,STOMATUS,70390.0,2000.0,NaN,NaN,NaN,NaN


In [7]:
# Verificando a quantidade de estabelecimentos em Brasília

df_estab_df.shape

(8605, 9)

In [8]:
# Verificando informações dos estabelecimentos em Brasília

df_estab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8605 entries, 275936 to 304534
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       8605 non-null   int64  
 1   nm_razao_social          8605 non-null   object 
 2   nm_fantasia              8605 non-null   object 
 3   id_cep                   8605 non-null   float64
 4   id_natureza_juridica     8605 non-null   float64
 5   id_tipo_estabelecimento  4322 non-null   float64
 6   id_atividade_principal   4322 non-null   float64
 7   lat                      0 non-null      float64
 8   lon                      0 non-null      float64
dtypes: float64(6), int64(1), object(2)
memory usage: 672.3+ KB


In [9]:
# Verificando informações dos estabelecimentos em Brasília

df_estab_df.describe()

,id,id_cep,id_natureza_juridica,id_tipo_estabelecimento,id_atividade_principal,lat,lon
count,8.605000e+03,8605.000000,8605.000000,4322.000000,4322.000000,0.0,0.0
mean,5.805224e+06,71011.669030,2338.457641,14.983572,2.813975,NaN,NaN
std,2.949186e+06,862.025354,761.785007,4.023417,4.330920,NaN,NaN
min,3.654000e+03,70002.000000,1000.000000,0.000000,1.000000,NaN,NaN
25%,3.262634e+06,70390.000000,2062.000000,16.000000,1.000000,NaN,NaN
50%,6.239307e+06,70710.000000,2062.000000,16.000000,1.000000,NaN,NaN
75%,9.054723e+06,71908.000000,2240.000000,16.000000,2.000000,NaN,NaN
max,9.999841e+06,73499.000000,4014.000000,25.000000,27.000000,NaN,NaN


In [10]:
# Dropando colunas com muitos valores ausentes

df_estab_df.drop(columns = ['id_tipo_estabelecimento','id_atividade_principal','lat','lon'],axis=1,inplace=True)
df_estab_df.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica
275936,3266303,RONILDA SAUL HAAS,RONILDA SAUL HAAS,70100.0,4000.0
276064,3106632,FLAVIA DE PASSOS,FLAVIA DE PASSOS,70100.0,4000.0
276308,3464857,CLINICA CONIC LTDA,ECOGRAFIAS CONIC,70100.0,2000.0
277135,7966946,LIFE ODONTOLOGIA LTDA,LIFE ODONTOLOGIA,70200.0,2062.0
277573,3522032,STOMATUS ODONTOLOGIA LTDA,STOMATUS,70390.0,2000.0


In [11]:
# Coletando dados dos estabelecimentos que foram postos de vacinação da base geral

df_estab_vacinacao = pd.read_sql_query("SELECT DISTINCT estabelecimento_id FROM pos.tbl_vacinacao", postgreSQLConnection)
df_estab_vacinacao.rename(columns={"estabelecimento_id": "id"},inplace=True)
df_estab_vacinacao.head()

,id
0,2496666
1,2442973
2,2718979
3,6039111
4,6822037


In [12]:
# Merge do dataframe de estabelecimentos do DF e dos postos de vacinação do Brasil para achar os estabelecimentos que foram postos de vacinação no DF

df_postos_df = pd.merge(df_estab_df,df_estab_vacinacao)
df_postos_df.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica
0,6268269,UBS 1 ITAPOA,UBS 1 ITAPOA,71593.0,1023.0
1,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,70160.0,1040.0
2,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200.0,1023.0
3,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203.0,1023.0
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,70310.0,1015.0


In [13]:
# Verificando quantidade de postos de vacinação no DF

df_postos_df.shape

(97, 5)

In [14]:
# Pelo id_cep, vamos pegar a lat e long da tabela de ceps

df_ceps_df.drop(columns='id_cidade',inplace=True)
df_ceps_df.rename(columns={"id":"id_cep"},inplace=True)
df_postos_df = pd.merge(df_postos_df,df_ceps_df,on='id_cep')
df_postos_df.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon
0,6268269,UBS 1 ITAPOA,UBS 1 ITAPOA,71593.0,1023.0,-15.790914,-48.071726
1,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,70160.0,1040.0,-15.800669,-47.861282
2,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200.0,1023.0,-15.821936,-47.890604
3,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203.0,1023.0,-15.823910,-47.897709
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,70310.0,1015.0,-15.798626,-47.885490


In [15]:
# Falta tratar a informação de natureza jurídica. Vamos puxar a tabela "natureza juridica"

df_nat_jur = pd.read_sql_query("SELECT * FROM pos.tbl_natureza_juridica", postgreSQLConnection)
df_nat_jur.rename(columns={"id":"id_natureza_juridica"},inplace=True)
df_nat_jur.head()

,id_natureza_juridica,natureza_juridica
0,1228,CONSORCIO PUBLICO DE DIREITO PRIVADO
1,1236,ESTADO OU DISTRITO FEDERAL
2,1244,MUNICIPIO
3,1252,FUNDACAO PUBLICA DE DIREITO PRIVADO FEDERAL
4,1260,FUNDACAO PUBLICA DE DIREITO PRIVADO ESTADUAL O...


In [16]:
# Merge da info de natureza juridica dos estabelecimentos

df_postos_df = pd.merge(df_postos_df,df_nat_jur,on='id_natureza_juridica')
df_postos_df.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica
0,6268269,UBS 1 ITAPOA,UBS 1 ITAPOA,71593.0,1023.0,-15.790914,-48.071726,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200.0,1023.0,-15.821936,-47.890604,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203.0,1023.0,-15.823910,-47.897709,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
3,5117666,UNIDADE BASICA DE SAUDE N 5 ASA NORTE,UBS 5 GRANJA DO TORTO,70636.0,1023.0,-15.836922,-47.932855,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
4,10758,UNIDADE BASICA DE SAUDE N 2 CRUZEIRO,UBS 2 CRUZEIRO,70640.0,1023.0,-15.789750,-47.939987,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...


In [17]:
#Algumas informações dos postos do DF

df_postos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    97 non-null     int64  
 1   nm_razao_social       97 non-null     object 
 2   nm_fantasia           97 non-null     object 
 3   id_cep                97 non-null     float64
 4   id_natureza_juridica  97 non-null     float64
 5   lat                   97 non-null     float64
 6   lon                   97 non-null     float64
 7   natureza_juridica     97 non-null     object 
dtypes: float64(4), int64(1), object(3)
memory usage: 6.8+ KB


In [18]:
# Analisando os dados de natureza jurídica
df_postos_df.natureza_juridica.value_counts()

ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU DO DISTRITO FEDERAL    88
ORGAO PUBLICO DO PODER EXECUTIVO FEDERAL                             4
SERVICO SOCIAL AUTONOMO                                              2
ORGAO PUBLICO DO PODER LEGISLATIVO FEDERAL                           1
FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL                             1
FUNDACAO FEDERAL                                                     1
Name: natureza_juridica, dtype: int64

In [19]:
# Verificando todos os estabelecimentos filtrados no nosso dataset

pd.set_option("max_rows", None)
df_postos_df.nm_fantasia

0                                    UBS 1 ITAPOA
1                                  UBS 01 ASA SUL
2      HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB
3                           UBS 5 GRANJA DO TORTO
4                                  UBS 2 CRUZEIRO
5                                            HRAN
6                                 UBS 2 ASA NORTE
7                                 UBS 1 ASA NORTE
8                   UBS 3 ASA NORTE VILA PLANALTO
9                    HOSPITAL REGIONAL DO GUARA I
10                                   UBS 01 GUARA
11                       UBS 04 GUARA LUCIO COSTA
12                              UBS 01 ESTRUTURAL
13                                   UBS 02 GUARA
14                                   UBS 03 GUARA
15                               UBS 1 LAGO NORTE
16                                   UBS 1 VARJAO
17                           UBS 3 PARANOA PARQUE
18                                            HRL
19                                  UBS 1 PARANOA


In [20]:
_# Importando o folium e o mapa do df
import folium as fl

m = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

In [21]:
#Geopandas
import geopandas

df_postos_geo = geopandas.GeoDataFrame(df_postos_df, geometry = geopandas.points_from_xy(df_postos_df.lon, df_postos_df.lat))
df_postos_geo.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
0,6268269,UBS 1 ITAPOA,UBS 1 ITAPOA,71593.0,1023.0,-15.790914,-48.071726,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-48.07173 -15.79091)
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200.0,1023.0,-15.821936,-47.890604,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.89060 -15.82194)
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203.0,1023.0,-15.823910,-47.897709,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.89771 -15.82391)
3,5117666,UNIDADE BASICA DE SAUDE N 5 ASA NORTE,UBS 5 GRANJA DO TORTO,70636.0,1023.0,-15.836922,-47.932855,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.93285 -15.83692)
4,10758,UNIDADE BASICA DE SAUDE N 2 CRUZEIRO,UBS 2 CRUZEIRO,70640.0,1023.0,-15.789750,-47.939987,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.93999 -15.78975)


In [22]:
# Criando marcadores
for i in range(0,len(df_postos_geo)):
  fl.Marker([df_postos_geo.loc[i,'lat'],df_postos_geo.loc[i,'lon']],tooltip= df_postos_geo.loc[i,'nm_fantasia']).add_to(m)

In [23]:
# Visualização do mapa

m

In [24]:
# Algumas UBS estão bem distantes da área do DF. Como são 5 casos, vale a pena olhar caso a caso.

df_postos_geo.loc[(df_postos_geo['nm_fantasia'] == "UBS 1 GAMA")]

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
60,10820,UBS 1 GAMA,UBS 1 GAMA,72415.0,1023.0,-15.288931,-48.594156,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-48.59416 -15.28893)


In [25]:
df_postos_geo.loc[(df_postos_geo['nm_fantasia'] == "UBS 2 GAMA")]

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
59,10839,UBS 2 GAMA,UBS 2 GAMA,72410.0,1023.0,-15.100514,-48.910861,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-48.91086 -15.10051)


In [26]:
df_postos_geo.loc[(df_postos_geo['nm_fantasia'] == "UBS 3 GAMA")]

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
63,10847,UBS 3 GAMA,UBS 3 GAMA,72457.0,1023.0,-17.343665,-47.186688,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.18669 -17.34366)


In [27]:
df_postos_geo.loc[(df_postos_geo['nm_fantasia'] == "UBS 1 ENGENHO VELHO FERCAL")]

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
83,2804360,UNIDADE BASICA DE SAUDE N 1 FERCAL,UBS 1 ENGENHO VELHO FERCAL,73150.0,1023.0,-16.60344,-47.933089,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-47.93309 -16.60344)


In [28]:
df_postos_geo.loc[(df_postos_geo['nm_fantasia'] == "UBS 9 ENGENHO DAS LAGES GAMA")]

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica,geometry
62,2779404,UBS 9 ENGENHO DAS LAGES GAMA,UBS 9 ENGENHO DAS LAGES GAMA,72426.0,1023.0,-15.751707,-48.269996,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,POINT (-48.27000 -15.75171)


In [29]:
#Verificando os dados na tabela da secretária de saúde do DF e vendo as USB, foi notado que os CEPS estão ok, mas os lat/lon estão bem diferentes. 
# http://www.saude.df.gov.br/wp-conteudo/uploads/joomla/238bab226de40145ad9645523074e6ce.pdf
# Será feita alteração no dataframe df_postos_df e no dataframe df_postos_geo

indices = [60,59,63,62,83]
lat_alt = [-16.020679,-16.036695,-16.005841,-16.04388,-15.600348]
lon_alt = [-48.084848,-48.06098,-48.052746,-48.254283,-47.8715]

for i, la, lo in zip(indices,lat_alt, lon_alt):
  df_postos_df.iloc[i,5] = la
  df_postos_geo.iloc[i,5] = la
  df_postos_df.iloc[i,6] = lo
  df_postos_geo.iloc[i,6] = lo

In [30]:
# Teste para ver se o loop anterior deu certo
df_postos_df.iloc[60,]

id                                                                  10820
nm_razao_social                                                UBS 1 GAMA
nm_fantasia                                                    UBS 1 GAMA
id_cep                                                              72415
id_natureza_juridica                                                 1023
lat                                                              -16.0207
lon                                                              -48.0848
natureza_juridica       ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
geometry                    POINT (-48.59415550889641 -15.28893084361635)
Name: 60, dtype: object

In [31]:
# Plotando novamente as UBSs

m1 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

for i in range(0,len(df_postos_geo)):
  fl.Marker([df_postos_geo.loc[i,'lat'],df_postos_geo.loc[i,'lon']],tooltip= df_postos_geo.loc[i,'nm_fantasia']).add_to(m1)

m1

In [32]:
# Análise da tbl de vacinação Brasil

dfVacinacao = pd.read_sql_query('SELECT hash_paciente,estabelecimento_id,data_aplicacao,dose_declarada FROM pos.tbl_vacinacao', postgreSQLConnection)
dfVacinacao.head()

,hash_paciente,estabelecimento_id,data_aplicacao,dose_declarada
0,78ded39490c9ebe2e3db77999d930fdb428e9c096563b4...,2377691,2021-03-18,2.0
1,a9a746a10cd1d8ed559282d63d76f22d101894b34eb64b...,6716504,2021-03-17,1.0
2,ed6bcba7f18c7bde101966a4d13867a7a937c59c12b259...,5950066,2021-03-18,1.0
3,8c57f8a29e3dce1dd8aff19fc967536068d21b0d62d9af...,2269554,2021-03-18,1.0
4,f3444e51a5d551d4223b8c04c10b633bd5d1a472cd763f...,6495583,2021-03-16,1.0


In [33]:
# Filtro da tbl de vacinacao apenas pro DF

postos = df_postos_df.id
dfVacinacao = dfVacinacao.loc[dfVacinacao['estabelecimento_id'].isin(postos)]

In [34]:
#Verificando o tamanho da tabela de vacinação do Df
dfVacinacao.shape

(243886, 4)

In [35]:
# Olhando vacinação no DF
dfVacinacao.head()

,hash_paciente,estabelecimento_id,data_aplicacao,dose_declarada
147,95e0404963b2e21c21f8d5afae33278c5dbb4559266816...,9056963,2021-03-03,1.0
164,b4cddad9830ed0c67c51c0ead1984ff882c55b6bcac239...,11010,2021-03-18,1.0
221,0185a60c6d6b8f31e443addd0ac1b3f4c6fe16a05b24de...,10863,2021-03-18,1.0
247,971aa17ab21f90e5a030a7369ec8e39781d7cac4900b68...,3144607,2021-03-19,1.0
272,e30c461f0408df147b365e8879a9736f94326ca21c15ee...,11010,2021-03-18,1.0


In [36]:
# Análise da tbl paciente

dfPaciente = pd.read_sql_query('SELECT hash,id_cidade,id_cep,id_grupo FROM pos.tbl_paciente WHERE id_cidade = 5300108', postgreSQLConnection)
dfPaciente.head()

,hash,id_cidade,id_cep,id_grupo
0,a7d7709102b06f450f3006dcbb0eb0e8e68c382ae9f0cf...,5300108,NaN,2.0
1,a7d78b6f891f0892bf357bf2c9c8f01f70fd1e1415a6fd...,5300108,72210.0,9.0
2,a7d7a9f04185fb4a9a07ace4722cf330585a2ebc6e7a50...,5300108,70772.0,9.0
3,a7d7b6902527f51c8d706d6df608d3b71079344aaffca3...,5300108,71020.0,2.0
4,a7d868bc37bdabbfefd6934b6f8f5762ce22b818702ed0...,5300108,70347.0,2.0


In [37]:
# Merge das tabelas de vacinação e paciente no hash do paciente para juntarmos as informações de endereço do estabelecimento com endereço do paciente

dfVacinacao.rename(columns={"hash_paciente":"hash"},inplace=True)
dfMerged = pd.merge(dfVacinacao,dfPaciente,on='hash')
dfMerged.head()

,hash,estabelecimento_id,data_aplicacao,dose_declarada,id_cidade,id_cep,id_grupo
0,95e0404963b2e21c21f8d5afae33278c5dbb4559266816...,9056963,2021-03-03,1.0,5300108,70673.0,2.0
1,b4cddad9830ed0c67c51c0ead1984ff882c55b6bcac239...,11010,2021-03-18,1.0,5300108,NaN,2.0
2,0185a60c6d6b8f31e443addd0ac1b3f4c6fe16a05b24de...,10863,2021-03-18,1.0,5300108,72460.0,2.0
3,971aa17ab21f90e5a030a7369ec8e39781d7cac4900b68...,3144607,2021-03-19,1.0,5300108,71884.0,2.0
4,2ff16b7715e0f5dea79ad8d7d9409a955ea06abdc34355...,11037,2021-03-18,1.0,5300108,NaN,2.0


In [38]:
# Tamanho do Merge
dfMerged.shape

(200875, 7)

In [39]:
# Checando as informações
dfMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200875 entries, 0 to 200874
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   hash                200875 non-null  object 
 1   estabelecimento_id  200875 non-null  int64  
 2   data_aplicacao      200875 non-null  object 
 3   dose_declarada      200875 non-null  float64
 4   id_cidade           200875 non-null  int64  
 5   id_cep              141593 non-null  float64
 6   id_grupo            200787 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 12.3+ MB


In [40]:
# Verificando quantidade de valores nulos de CEP
print(dfMerged.id_cep.isna().sum()/len(dfMerged))

0.29511885500933416


In [41]:
# Quase 30% dos vacinados do DF não possuem número do CEP da residência na base de dados
# Vamos dropar esses valores

dfMerged = dfMerged.dropna()
dfMerged.shape

(141521, 7)

In [ ]:
#Verificando o começo do dataset
dfMerged.head()

In [44]:
# Retirada das pessoas que tomaram 2 doses para manter no dataset apenas pacientes unicos

dfMerged = dfMerged[dfMerged.dose_declarada == 1]
dfMerged.dose_declarada.value_counts()

1.0    113948
Name: dose_declarada, dtype: int64

In [45]:
# Verificação do shape
dfMerged.shape

(113948, 7)

In [46]:
# Agrupando os pacientes de acordo com o estabelecimento id e cep da residência
dfMerged_group = dfMerged[['estabelecimento_id','id_cep']].copy()
dfMerged_group = dfMerged_group.groupby(['estabelecimento_id','id_cep']).size().reset_index(name='contador')
dfMerged_group.head(10)

,estabelecimento_id,id_cep,contador
0,10456,4548.0,1
1,10456,55680.0,1
2,10456,64280.0,1
3,10456,65605.0,1
4,10456,70002.0,4
5,10456,70040.0,1
6,10456,70050.0,1
7,10456,70089.0,1
8,10456,70210.0,1
9,10456,70232.0,1


In [47]:
# Valor máximo do contador
dfMerged_group.contador.max()

934

In [48]:
#Puxar lat e long dos estabelecimentos
df_postos_df.rename(columns={"id":"estabelecimento_id"},inplace=True)
dfMerged_group = pd.merge(dfMerged_group,df_postos_df[['lat','lon','estabelecimento_id']],on='estabelecimento_id')
dfMerged_group.head()

,estabelecimento_id,id_cep,contador,lat,lon
0,10456,4548.0,1,-15.800697,-47.889197
1,10456,55680.0,1,-15.800697,-47.889197
2,10456,64280.0,1,-15.800697,-47.889197
3,10456,65605.0,1,-15.800697,-47.889197
4,10456,70002.0,4,-15.800697,-47.889197


In [49]:
#Renomeando as colunas lat e long
dfMerged_group.rename(columns={"lat":"lat_estab","lon":"lon_estab"},inplace=True)

In [50]:
# Puxar lat e long das residências dos pacientes
dfMerged_group = pd.merge(dfMerged_group,df_ceps_df)
dfMerged_group.rename(columns={"lat":"lat_resid","lon":"lon_resid"},inplace=True)
dfMerged_group.head()

,estabelecimento_id,id_cep,contador,lat_estab,lon_estab,lat_resid,lon_resid
0,10456,70002.0,4,-15.800697,-47.889197,-15.79084,-47.879656
1,10464,70002.0,2,-15.767055,-47.876769,-15.79084,-47.879656
2,10472,70002.0,5,-15.871168,-47.980211,-15.79084,-47.879656
3,10480,70002.0,2,-15.825485,-48.095874,-15.79084,-47.879656
4,10499,70002.0,8,-15.808742,-48.060183,-15.79084,-47.879656


In [51]:
# Checando informações
dfMerged_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14840 entries, 0 to 14839
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   estabelecimento_id  14840 non-null  int64  
 1   id_cep              14840 non-null  float64
 2   contador            14840 non-null  int64  
 3   lat_estab           14840 non-null  float64
 4   lon_estab           14840 non-null  float64
 5   lat_resid           14840 non-null  float64
 6   lon_resid           14840 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 927.5 KB


In [ ]:
# Medindo distância entre a residência dos pacientes até o posto de vacinação
import haversine as hs



In [ ]:
# Fechando conexão
#postgreSQLConnection.close()